Building off of my earlier notebooks, I would like to be able to render the collatz tree in a way that's easier to read.

Given a node $x$, its children are the given by $\frac{2^ix - 1}{3}$ where $2^ix \equiv_3 1$ and $i>0$.  Note that if $x\equiv_30$ then node is a leaf.  The root is $x=1$.

Recall that $2^{\text{even}}\equiv_3 1$ and $2^{\text{odd}}\equiv_3 2$, so if $x\equiv_31$ then $i$ even and if $x\equiv_3 2$ then $i$ odd.

In [1]:
def get_children(x, n):
    """Returns list of first n children of x"""
    if x % 3 == 0:
        return []
    elif x % 3 == 1:
        evens = [2 ** (2 * i) * x for i in range(1, n+1)]
        return [(x - 1) // 3 for x in evens]
    else:
        evens = [2 ** (2 * i + 1) * x for i in range(0, n)]
        return [(x - 1) // 3 for x in evens]

In [2]:
get_children(1, 5), get_children(5, 5), get_children(85, 5), get_children(7, 5), get_children(9,5)

([1, 5, 21, 85, 341],
 [3, 13, 53, 213, 853],
 [113, 453, 1813, 7253, 29013],
 [9, 37, 149, 597, 2389],
 [])

In [3]:
def build_tree_helper(x, n, d, max_d, b):
    import collatz
    ret = {}
    ret['name'] = '{}, {}'.format(x % 3, collatz.basen(x, b))
    if d == max_d:
        return ret
    children = get_children(x, n)
    if x == 1:
        children = children[1:]
    if children:
        ret['children'] = [build_tree_helper(x, n, d + 1, max_d, b) for x in children]
    return ret

def build_tree(n, d, b):
    return build_tree_helper(1, n, 1, d, b)

In [4]:
collatz_tree = build_tree(6, 7, 2)

In [5]:
import json
with open('collatz.json', 'w') as fh:
    fh.write(json.dumps(collatz_tree))

In [6]:
width = 2000
height= 500

def replace(_tmpl, **kwargs):
    for k, v in kwargs.items():
        _tmpl = _tmpl.replace('{{{{{}}}}}'.format(k), str(v))
    return _tmpl
    

with open('tree.js.tmpl') as fh:
    tmpl = ''.join(fh.readlines())
    js_src = replace(tmpl, width=width, height=height, filename='collatz.json')

In [7]:
with open('tree.html') as fh:
    html_src = ''.join(fh.readlines())
html_src = replace(html_src, width=width)

In [8]:
html = html_src + '<script>' + js_src + '</script>'
from IPython.core.display import HTML
HTML(html)

What I am hoping for here is that we can whittle down the numbers that matter and then show that the collatz sequence enumerates this meaningful set.

We can see some patterns from the visualization.

**pattern 1**: Looking at the base2 repr, $101X$ takes the same number of steps as $1X$.

**proof**: Let $Y$ be the parent of $1X$ and note that $101X = 4 \cdot 1X + 1$. 

If $Y \equiv_3 1$, then $1X = \frac{2^{2i}Y - 1}{3}$ where $i \geq 1$. 

$\begin{align}
101X &= 2^2 \big(\frac{2^{2i}Y - 1}{3}\big) + 1 &= 2^2 \big(\frac{2^{2i}Y - 1}{3}\big) + \frac{3}{3}
&= \frac{2^{2(i+1)}Y - 4 + 3}{3} &= \frac{2^{2(i+1)}Y - 1}{3}
\end{align}$

So $Y$ is the parent of $101X$.

If $Y \equiv_3 2$, then $1X = \frac{2^{2i+1}Y - 1}{3}$ where $i \geq 0$ and the proof is analogous.


**pattern 2**: If $1Y \equiv_3 1$, then $101Y \equiv_3 2$ and if first child of $1Y$ is $1X$, then first child of $101Y$ is $11X$. 

**proof**: Note that first child of $1Y$ is $\frac{2^21Y - 1}{3}$ and the first child of $101Y$ is $\frac{2\cdot 101Y -1}{3}$. We see that

$
\begin{align}
\frac{2\cdot 101Y -1}{3} = \frac{2 (2^2 1Y + 1) - 1}{3} = \frac{2 \cdot 2^21Y + 2 - 1}{3} = \frac{2 \cdot 2^21Y + 1}{3} = \frac{2 \cdot 2^21Y - 2 + 1 + 2}{3} = \frac{2\cdot(2^21Y - 1) + 3}{3} = 2\big[\frac{2^21Y - 1}{3}\big] + 1
\end{align}
$

This means that first child of $101Y$ is twice the first child of $1Y$ plus 1, which means it is $11X$.

**pattern 3**: 
- If $Y \equiv_3 1$ and first child of $1Y$ is $1X$, then first child of $1010101Y$ is $1000111X$.
- If $Y \equiv_3 2$ and first child of $1Y$ is $11X$, then first child of $1010101Y$ is $11000111X$.

**proof**: $1010101Y \equiv_3 1Y$. (TODO)

**pattern 4**: If $1Y \equiv_3 1$ and $101Y$ is $2X$ in base 3, then first child of $1Y$ is $X$ in base 3.

**proof**: First child of $1Y$ is $\frac{2^21Y - 1}{3}$. We see that
$
\begin{align}
2X = 3\cdot X + 2 = 3 \frac{2^21Y - 1}{3} + 2 = 2^21Y - 1 + 2 = 2^21Y + 2 = 101Y 
\end{align}
$

**pattern 5**: Numbers of the form $1\cdots101 \equiv_3 2$

**proof**: Simple induction.  Base case is 101.  Inductive step: Assume true for $X=1\cdots101$ with $k$ ones. Then $k+1$ ones is equivalent to $2\cdot X + 1 \equiv_3 2\cdot2 + 1 \equiv_3 2$.


Given these patterns, we can "reduce" any number to the first one in its level.  So we only have to consider numbers that are the first in their levels.

Strings of 1s with an odd number of ones.


# future work
Some ideas for how to proceed from here:

- Let's look at the sequence of first children (with a child) 1 -> 101 -> 1011 -> 10001 -> 1101 -> 111 -> ... Can we characterize this sequence?
- Take any number. Apply the patterns above.  We should reach a number that is on the "generator" sequence.  If we can do that we're done because the generator sequence leads to 1.

# limitations
These patterns are localized.  They just involve the immediate parent.  We can eliminate all of the extraneous '10', but it's not clear what to do next after that.  We have to look at the parent's modulus or what not. I know that now the first 3 bits must not be 101. Cannot start with 0 either.  So it must be 111, 110, or 100.
```
Case 101:
    apply pattern 1.

Case 100:
    if first 7 bits are 1000111: 
        apply pattern 3.
    else: 
        don't know.

Case 111:
    if parent % 3 == 2:
        if parent starts with 101:
            apply pattern 2.
        else:
            don't know.
    else:
        apply pattern 4.
 
Case 110:
    if parent % 3 == 2:
      if parent starts with 101:
          apply pattern 2.
      else:
          don't know.
    else:
      apply pattern 4.
```

The collatz is hard because the base2 representation can grow but at some point there will be large cancelations as a large power of 2 appears in the prime factorization.  